In [3]:
# Dependencies
import requests
import json
from pprint import pprint
import pandas as pd
import os
import csv
import numpy as np
import time 
import tweepy
from config import consumer_key, consumer_secret, access_token, access_token_secret
from datetime import datetime
import math

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

path = "resources/Gazetteer_2017_counties.csv"
GazCo = pd.read_csv(path, encoding = "latin1")
df_geo = pd.DataFrame(data=GazCo)
df_geo.head(2)

,USPS,GEOID,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001,161526,Autauga County,1.539615e+09,25744269,594.449,9.940,32.532237,-86.646440
1,AL,1003,161527,Baldwin County,4.117606e+09,1133109409,1589.817,437.496,30.659218,-87.746067


In [4]:
output_list = pd.DataFrame(columns=['created_at','account_location','text_tweet', 'place_type',
                                    'country_code', 'place_name_long', 'place_name_short', 'latitude', 'longitude',
                                   'coordinate_box1_lat','coordinate_box2_lat','coordinate_box3_lat','coordinate_box4_lat',
                                   'coordinate_box1_lon','coordinate_box2_lon','coordinate_box3_lon','coordinate_box4_lon'])

# "Real Person" Filters
min_tweets = 2
max_tweets = 50000
max_followers = 8000
max_following = 8000
lang = "en"

# Set other parameters
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify= True, parser=tweepy.parsers.JSONParser())
df_geo = df_geo[['INTPTLAT','INTPTLONG', 'ALAND_SQMI']]
entry = 0
unique_ids = []
search_term = ('immigration OR Immigration OR immigrant OR immigrants OR foreigner OR noncitizen OR undocumented OR non-citizen OR permanent resident')
oldest_tweet = None

In [7]:
# Super Twitter Loop
for (idx, row) in df_geo.iterrows():
    #entry += 1 
    latitude = (row.loc['INTPTLAT'])
    longitude = (row.loc['INTPTLONG'])
    LAND_SQMI = (row.loc['ALAND_SQMI'])
    distance = math.sqrt(LAND_SQMI)/2
    for x in range(30):
        try:
            public_tweets = api.search(q=search_term,
                                     geocode=f"{latitude},{longitude},{distance}mi",
                                     count=100,
                                    result_type="recent",
                                     max_id=oldest_tweet)
            for tweet in public_tweets["statuses"]:
                tweet_id = tweet_id = tweet["id"]
            # Use filters to check if user meets conditions
                if (tweet["user"]["followers_count"] < max_followers and
                    tweet["user"]["statuses_count"] > min_tweets and
                    tweet["user"]["statuses_count"] < max_tweets and
                    tweet["user"]["friends_count"] < max_following and
                    tweet["user"]["lang"] == lang):
            #Read information
                    if tweet_id not in unique_ids:
                        created_at = tweet["created_at"]
                        pprint(created_at)
                        account_location = tweet['user']["location"]
                        pprint(account_location)
                        text_tweet = tweet["text"]
                        pprint(text_tweet)
                        my_place = tweet["place"]
                        pprint(my_place)
                        if my_place is None: 
                            place_name_short = np.nan
                            place_name_long = np.nan
                            place_type = np.nan
                            country_code = np.nan
                            coordinate_box1_lat = np.nan
                            coordinate_box2_lat = np.nan
                            coordinate_box3_lat = np.nan
                            coordinate_box4_lat = np.nan
                            coordinate_box1_lon = np.nan
                            coordinate_box2_lon = np.nan
                            coordinate_box3_lon = np.nan
                            coordinate_box4_lon = np.nan
                        else:
                            place_name_short = tweet["place"]['name'][0]
                            place_name_long = tweet["place"]['full_name'][0]
                            place_type = tweet["place"]['place_type'][0]
                            country_code = tweet["place"]['country_code'][0]
                            coordinate_box1_lat = tweet["place"]['bounding_box']['coordinates'][0][0][0]
                            coordinate_box1_lon = tweet["place"]['bounding_box']['coordinates'][0][0][1]
                            coordinate_box2_lat = tweet["place"]['bounding_box']['coordinates'][0][1][0]
                            coordinate_box2_lon = tweet["place"]['bounding_box']['coordinates'][0][1][1]
                            coordinate_box3_lat = tweet["place"]['bounding_box']['coordinates'][0][2][0]
                            coordinate_box3_lon = tweet["place"]['bounding_box']['coordinates'][0][2][1]
                            coordinate_box4_lat = tweet["place"]['bounding_box']['coordinates'][0][3][0]
                            coordinate_box4_lon = tweet["place"]['bounding_box']['coordinates'][0][3][1]
                        created_at =datetime.strptime(created_at, "%a %b %d %H:%M:%S %z %Y")
                        output_list.at[idx, 'latitude'] = latitude
                        output_list.at[idx, 'longitude'] = longitude
                        output_list.at[idx, 'created_at'] = created_at
                        output_list.at[idx, 'text_tweet'] = text_tweet
                        output_list.at[idx, 'account_location'] = account_location
                        output_list.at[idx, 'place_name_short'] = place_name_short         
                        output_list.at[idx, 'place_name_long'] = place_name_long         
                        output_list.at[idx, 'place_type'] = place_type         
                        output_list.at[idx, 'coordinate_box1_lat'] = coordinate_box1_lat         
                        output_list.at[idx, 'coordinate_box2_lat'] = coordinate_box2_lat         
                        output_list.at[idx, 'coordinate_box3_lat'] = coordinate_box3_lat         
                        output_list.at[idx, 'coordinate_box4_lat'] = coordinate_box4_lat
                        output_list.at[idx, 'coordinate_box1_lon'] = coordinate_box1_lon         
                        output_list.at[idx, 'coordinate_box2_lon'] = coordinate_box2_lon         
                        output_list.at[idx, 'coordinate_box3_lon'] = coordinate_box3_lon         
                        output_list.at[idx, 'coordinate_box4_lon'] = coordinate_box4_lon         
        except Exception as e:
            print(e)
            time.sleep(5)
            print("restarting...")
            pass

Rate limit reached. Sleeping for: 700


Failed to send request: ('Connection aborted.', OSError("(54, 'ECONNRESET')",))
restarting...
Failed to send request: ("bad handshake: SysCallError(54, 'ECONNRESET')",)
restarting...
'Wed Apr 11 22:03:45 +0000 2018'
'Pennsylvania, USA'
('Citizens Win In Arizona, Illegal Immigrants Cannot Receive Resident Tuition '
 'https://t.co/J6dFSUjpLf via @ConDailyPost')
None
'Fri Apr 06 14:08:59 +0000 2018'
'Pennsylvania, USA'
("@cmdeb I've had them in England and Scotland, but the best I've had were "
 'homemade in....Pennsylvania (to be fair, h… https://t.co/BF8cRrikDQ')
None
'Fri Apr 06 06:46:38 +0000 2018'
'Pennsylvania, USA'
('@RightlyNews Wonderful news from a life long Pennsylvania resident. Former '
 'Democrat (embarrassed by that) Finally b… https://t.co/Py8lC19zH3')
None
'Wed Apr 11 22:03:45 +0000 2018'
'Pennsylvania, USA'
('Citizens Win In Arizona, Illegal Immigrants Cannot Receive Resident Tuition '
 'https://t.co/J6dFSUjpLf via @ConDailyPost')
None
'Fri Apr 06 14:08:59 +0000 2018'
'P

KeyboardInterrupt: 

In [8]:
output_list.head(200)

,created_at,account_location,text_tweet,place_type,country_code,place_name_long,place_name_short,latitude,longitude,coordinate_box1_lat,coordinate_box2_lat,coordinate_box3_lat,coordinate_box4_lat,coordinate_box1_lon,coordinate_box2_lon,coordinate_box3_lon,coordinate_box4_lon
1,2018-04-06 06:46:38+00:00,"Pennsylvania, USA",@RightlyNews Wonderful news from a life long P...,NaN,NaN,NaN,NaN,30.6592,-87.7461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
output_test = output_list
#.dropna(subset=['place_name_long'])

output_test.count()